# Assignement : Segmenting and Clustering Neighborhoods in Toronto

## 1. Create DataFrame

### Import libraries

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

# libraries to scrape data from wikipedia
import requests 
from bs4 import BeautifulSoup


print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /opt/anaconda3

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    certifi-2019.9.11          |           py37_0         147 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         860 KB

The following NEW packages will be INSTALLED:

### Scrape data from wikipedia

In [207]:
# get the HTML of the website
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

# read the source code and createa BeautifulSoup (soup)object 
soup = BeautifulSoup(website_url,'lxml')

# untag to view how the tags are nested in the document.
# print(soup.prettify())

In [171]:
# find class ‘wikitable sortable’ in the HTML script
My_table = soup.find('table',{'class':'wikitable sortable'})
My_table

<table class="wikitable sortable">
<tbody><tr>
<th>Postal Code
</th>
<th>Borough
</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A
</td>
<td>North York
</td>
<td>Parkwoods
</td></tr>
<tr>
<td>M4A
</td>
<td>North York
</td>
<td>Victoria Village
</td></tr>
<tr>
<td>M5A
</td>
<td>Downtown Toronto
</td>
<td>Regent Park, Harbourfront
</td></tr>
<tr>
<td>M6A
</td>
<td>North York
</td>
<td>Lawrence Manor, Lawrence Heights
</td></tr>
<tr>
<td>M7A
</td>
<td>Downtown Toronto
</td>
<td>Queen's Park, Ontario Provincial Government
</td></tr>
<tr>
<td>M8A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M9A
</td>
<td>Etobicoke
</td>
<td>Islington Avenue, Humber Valley Village
</td></tr>
<tr>
<td>M1B
</td>
<td>Scarborough
</td>
<td>Malvern, Rouge
</td></tr>
<tr>
<td>M2B
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3B
</td>
<td

In [172]:
# extract all the data within "td"
links = My_table.findAll('td')
links

[<td>M1A
 </td>, <td>Not assigned
 </td>, <td>Not assigned
 </td>, <td>M2A
 </td>, <td>Not assigned
 </td>, <td>Not assigned
 </td>, <td>M3A
 </td>, <td>North York
 </td>, <td>Parkwoods
 </td>, <td>M4A
 </td>, <td>North York
 </td>, <td>Victoria Village
 </td>, <td>M5A
 </td>, <td>Downtown Toronto
 </td>, <td>Regent Park, Harbourfront
 </td>, <td>M6A
 </td>, <td>North York
 </td>, <td>Lawrence Manor, Lawrence Heights
 </td>, <td>M7A
 </td>, <td>Downtown Toronto
 </td>, <td>Queen's Park, Ontario Provincial Government
 </td>, <td>M8A
 </td>, <td>Not assigned
 </td>, <td>Not assigned
 </td>, <td>M9A
 </td>, <td>Etobicoke
 </td>, <td>Islington Avenue, Humber Valley Village
 </td>, <td>M1B
 </td>, <td>Scarborough
 </td>, <td>Malvern, Rouge
 </td>, <td>M2B
 </td>, <td>Not assigned
 </td>, <td>Not assigned
 </td>, <td>M3B
 </td>, <td>North York
 </td>, <td>Don Mills
 </td>, <td>M4B
 </td>, <td>East York
 </td>, <td>Parkview Hill, Woodbine Gardens
 </td>, <td>M5B
 </td>, <td>Downtown Toronto
 

In [173]:
# Convert data in str
liste = []
for link in links:
    liste.append(str(link))

# extract data
liste_1 = []
for l in liste:
    liste_a=l.split(sep=">")
    a=liste_a[1]
    liste_b=a.split(sep="\n")
    b=liste_b[0] 
    liste_1.append(b)

In [174]:
# create range of numbers for later
stop_n=len(liste_1)+1
liste_num_1=list(range(1,stop_n,3))
liste_num_2=list(range(2,stop_n,3))
liste_num_3=list(range(3,stop_n,3))
# Create lists
PostalCode = []
Borough = []
Neighborhood = []
# Compteur
a=1
# Append data in the right list
for l in liste_1:
    if a in liste_num_1:
        PostalCode.append(l)
        a=a+1
    elif a in liste_num_2:
        Borough.append(l)
        a=a+1
    elif a in liste_num_3:
        Neighborhood.append(l)
        a=a+1

# create a dict with the 3 lists
data_nb = {'PostalCode': PostalCode, 'Borough': Borough, 'Neighborhood': Neighborhood}
# instantiate the dataframe with dict
neighborhoods = pd.DataFrame(data=data_nb)

In [175]:
# Get names of indexes for which column Borough has value Not assigne
indexNames = neighborhoods[ neighborhoods['Borough'] == "Not assigned" ].index
# Delete these row indexes from dataFrame
neighborhoods.drop(indexNames , inplace=True)

In [176]:
# Verify if there is duplicate in PostalCode
neighborhoods.duplicated(subset=['PostalCode'])

2      False
3      False
4      False
5      False
6      False
8      False
9      False
11     False
12     False
13     False
14     False
17     False
18     False
20     False
21     False
22     False
23     False
26     False
27     False
30     False
31     False
32     False
36     False
39     False
40     False
41     False
45     False
46     False
47     False
48     False
49     False
50     False
54     False
55     False
56     False
57     False
58     False
59     False
63     False
64     False
65     False
66     False
67     False
68     False
72     False
73     False
74     False
75     False
76     False
77     False
80     False
81     False
82     False
83     False
84     False
85     False
86     False
89     False
90     False
91     False
92     False
93     False
94     False
95     False
98     False
99     False
100    False
102    False
103    False
104    False
107    False
108    False
109    False
111    False
112    False
113    False
114    False

__There is no rows with the same Postal Code__

In [177]:
# Verify if there is Neighborhood with the value Not assigned
neighborhoods['Neighborhood'] == "Not assigned"

2      False
3      False
4      False
5      False
6      False
8      False
9      False
11     False
12     False
13     False
14     False
17     False
18     False
20     False
21     False
22     False
23     False
26     False
27     False
30     False
31     False
32     False
36     False
39     False
40     False
41     False
45     False
46     False
47     False
48     False
49     False
50     False
54     False
55     False
56     False
57     False
58     False
59     False
63     False
64     False
65     False
66     False
67     False
68     False
72     False
73     False
74     False
75     False
76     False
77     False
80     False
81     False
82     False
83     False
84     False
85     False
86     False
89     False
90     False
91     False
92     False
93     False
94     False
95     False
98     False
99     False
100    False
102    False
103    False
104    False
107    False
108    False
109    False
111    False
112    False
113    False
114    False

__There is no neighborhood with Not Assigned value__

In [208]:
neighborhoods.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.653482,-79.383935
3,M4A,North York,Victoria Village,43.653482,-79.383935
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.653482,-79.383935
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.653482,-79.383935
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.653482,-79.383935
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.653482,-79.383935
9,M1B,Scarborough,"Malvern, Rouge",43.653482,-79.383935
11,M3B,North York,Don Mills,43.653482,-79.383935
12,M4B,East York,"Parkview Hill, Woodbine Gardens",43.653482,-79.383935
13,M5B,Downtown Toronto,"Garden District, Ryerson",43.653482,-79.383935


In [178]:
# show number of rows 
a,b=neighborhoods.shape
print("My DataFrame has %a rows" %a)

My DataFrame has 103 rows


## 2. Add Latitude and longitude

### Initiation of a while loop for each postal code, to make sure that we get the coordinates for all of our neighborhoods

In [209]:
import geocoder # import geocoder
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

latitude = []
longitude = []

for postal_code in neighborhoods['PostalCode']:
    # initialize your variable to None
    location = None
    print(postal_code)
    a=0
    # loop until you get the coordinates
    while(location is None):
        print(a)
        geolocator = Nominatim(user_agent="ny_explorer")
        location = geolocator.geocode('{}, Toronto, Ontario'.format(postal_code))
        a=a+1
    latitude.append(location.latitude)
    longitude.append(location.longitude)

M3A
0
M4A
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14


KeyboardInterrupt: 

In [205]:
# verify that it is not working with M4A
location = None
address = 'M4A, Toronto, Canada'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print(latitude,longitude )

AttributeError: 'NoneType' object has no attribute 'latitude'

### Add data recolted in the dataframe

In [198]:
neighborhoods['Latitude'] = latitude 
neighborhoods['Longitude'] = longitude 

## 3. Make cluster

### Slice the original dataframe and create a new dataframe of the Toronto data.

In [188]:
toronto_data = neighborhoods.reset_index()
toronto_data.head()

,index,PostalCode,Borough,Neighborhood
0,2,M3A,North York,Parkwoods
1,3,M4A,North York,Victoria Village
2,4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,5,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Create a map of Toronto with neighborhoods superimposed on top.

In [199]:
# get latitude and longitude 
address = 'Toronto, Canada'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude,longitude )

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

43.6534817 -79.3839347


### Define Foursquare Credentials and Version

In [200]:
CLIENT_ID = 'M1504ORIFARIWEB405KNKN0TSEXFOXTKDDN3VWSX4WM1OE1O' # your Foursquare ID
CLIENT_SECRET = '1MQSZLE2L2ZCTDBIJFLLVROURUUYEQGNVBDCPD2B1PQATE2A' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: M1504ORIFARIWEB405KNKN0TSEXFOXTKDDN3VWSX4WM1OE1O
CLIENT_SECRET:1MQSZLE2L2ZCTDBIJFLLVROURUUYEQGNVBDCPD2B1PQATE2A


### Create a function to explore neighborhoods in Toronto

In [201]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Run the above function on each neighborhood and create a new dataframe called *toronto_venues*.

In [ ]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )



In [ ]:
### Analyze Each Neighborhood

In [ ]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [ ]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

Just make sure:

to add enough Markdown cells to explain what you decided to do and to report any observations you make.
to generate maps to visualize your neighborhoods and how they cluster together.
Once you are happy with your analysis, submit a link to the new Notebook on your Github repository. (3 marks)